In [1]:
import numpy as np
import pandas as pd
import keras.backend as K
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Reshape, Flatten, Dropout, Activation
from keras.models import Model, Sequential, load_model
from keras.layers.normalization import BatchNormalization
from keras import optimizers, callbacks
from skimage.transform import resize
from imageio import imread
from imgaug import augmenters as iaa

/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aldopedraza/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path_data = '../all/'

In [3]:
def preprocess_input(np_img):
    np_img = np_img / 255
    
    # randome data augmentation
    aug = np.random.choice([0, 1])
    if aug == True:
        seq = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
        np_img = seq.augment_image(np_img)
        
        # Zoom out randomly
        if np.random.choice([True, False]):
            seq = iaa.Sequential([iaa.Affine(scale={"x": (1.4, 1.4), "y": (1.4, 1.4)})])
            np_img = seq.augment_image(np_img)
        
    return np_img

In [4]:
def generator(path_images, labels, batch_size):
    count = 0
    while True:
        batch_features = []
        batch_labels = []
        for i in range(batch_size):
            # Read image
            name_img = labels[count][0] + '.jpg'
            img = imread(path_images + '/' + name_img)
            
            # Image preprocessing
            img = preprocess_input(img)

            label = labels[count][1:]

            batch_features.append(img)
            batch_labels.append(label)
            count += 1
            
            # Restart counter when it has reached the size 
            # of the data set
            if count == labels.shape[0] - 1:
                count = 0
                break
            
        yield np.array(batch_features), np.array(batch_labels)
            

In [5]:
def generator_predictions(path_images, list_names, batch_size):
    size_list = list_names.shape[0] - 1
    count = 0
    while True:
        batch_features = []
        for i in range(batch_size):
            # Read image
            name_img = list_names[count]
            img = imread(path_images + '/' + name_img)
            
            # Image preprocessing
            img = preprocess_input(img)

            batch_features.append(img)
            
            # Restart counter when it has reached the size 
            # of the data set
            if count == size_list:
                count = 0
                yield np.array(batch_features)
            count += 1
            
        yield np.array(batch_features)

In [6]:
def galaxy_cnn(input_size, output_size, drop_out=False, batch_norm=False, dense_size=1024):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_size))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(256, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(512, kernel_size=(3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if batch_norm:
        model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(dense_size, activation='relu'))
    if drop_out:
        model.add(Dropout(0.5))
    model.add(Dense(dense_size, activation='relu'))
    if drop_out:
        model.add(Dropout(0.5))
    model.add(Dense(output_size, activation='sigmoid'))
    return model

In [7]:
def rmse (y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

In [8]:
labels_train = np.load(path_data + 'y_train.npy')
labels_test = np.load(path_data + 'y_test.npy')
labels_val = np.load(path_data + 'y_val.npy')

In [11]:
dense_size = [1024, 2048]
drop_o = [True]
batch_n = [False, True]

In [12]:
batch_size = 32
steps = 43104 // batch_size 
steps_va = 9237 // batch_size
epochs = 20

In [11]:
for d_s in dense_size:
    for d_o in drop_o:
        for b_n in batch_n:
            model_name = 'model' + '_Dense_'+ str(d_s) + '_Drop_out_' + str(d_o) + '_Batch_n_' + str(b_n)
            print('Training model: ', model_name)
            # Set architecture
            opt = optimizers.Adam(lr=0.0001)
            model = galaxy_cnn((160, 160, 3), 37, d_o, b_n, d_s)
            model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse])
            tbCallBack = callbacks.TensorBoard(log_dir=path_data + './Graph/' + model_name, histogram_freq=0, write_graph=True, write_images=False)
            # Train the model
            model.fit_generator(generator(path_data + 'preprocess_img_train', labels_train, batch_size=batch_size), 
                    steps_per_epoch=steps, 
                    epochs=epochs, 
                    validation_data= generator(path_data + 'preprocess_img_val', labels_val, batch_size=batch_size),
                    validation_steps=steps_va, callbacks=[tbCallBack])
            # Evaluation over validation test
            evaluation = model.evaluate_generator(generator(path_data + 'preprocess_img_test', labels_test, batch_size=batch_size), steps=steps)
            print(model_name)
            print('Evaluation:', evaluation)
            # Save model after training
            model.save(path_data + 'weights/' + model_name + '_aug.h5')
            # Clear session to free memory
            K.clear_session()

Training model:  model_Dense_1024_Drop_out_True_Batch_n_False
Epoch 1/20
1347/1347 [==============================] - 144s 107ms/step - loss: 0.0256 - rmse: 0.1571 - val_loss: 0.0186 - val_rmse: 0.1359
Epoch 2/20
1347/1347 [==============================] - 132s 98ms/step - loss: 0.0185 - rmse: 0.1356 - val_loss: 0.0160 - val_rmse: 0.1262
Epoch 3/20
1347/1347 [==============================] - 131s 97ms/step - loss: 0.0157 - rmse: 0.1248 - val_loss: 0.0134 - val_rmse: 0.1155
Epoch 4/20
1347/1347 [==============================] - 135s 100ms/step - loss: 0.0138 - rmse: 0.1172 - val_loss: 0.0122 - val_rmse: 0.1101
Epoch 5/20
1347/1347 [==============================] - 135s 100ms/step - loss: 0.0127 - rmse: 0.1125 - val_loss: 0.0115 - val_rmse: 0.1070
Epoch 6/20
1347/1347 [==============================] - 135s 100ms/step - loss: 0.0121 - rmse: 0.1096 - val_loss: 0.0112 - val_rmse: 0.1053
Epoch 7/20
1347/1347 [==============================] - 135s 100ms/step - loss: 0.0116 - rmse: 0.107

1347/1347 [==============================] - 145s 108ms/step - loss: 0.0084 - rmse: 0.0914 - val_loss: 0.0091 - val_rmse: 0.0950
Epoch 17/20
1347/1347 [==============================] - 145s 108ms/step - loss: 0.0082 - rmse: 0.0905 - val_loss: 0.0091 - val_rmse: 0.0950
Epoch 18/20
1347/1347 [==============================] - 145s 108ms/step - loss: 0.0081 - rmse: 0.0899 - val_loss: 0.0090 - val_rmse: 0.0943
Epoch 19/20
1347/1347 [==============================] - 145s 108ms/step - loss: 0.0079 - rmse: 0.0889 - val_loss: 0.0087 - val_rmse: 0.0931
Epoch 20/20
1347/1347 [==============================] - 145s 108ms/step - loss: 0.0078 - rmse: 0.0882 - val_loss: 0.0089 - val_rmse: 0.0938
model_Dense_2048_Drop_out_True_Batch_n_False
Evaluation: [0.00889094145014219, 0.09391871578396142]
Training model:  model_Dense_2048_Drop_out_True_Batch_n_True
Epoch 1/20
1347/1347 [==============================] - 151s 112ms/step - loss: 0.0206 - rmse: 0.1418 - val_loss: 0.0147 - val_rmse: 0.1211
Epoch 

In [13]:
model_names = []
evaluations = []
for d_s in dense_size:
    for d_o in drop_o:
        for b_n in batch_n:
            model_name = 'model' + '_Dense_'+ str(d_s) + '_Drop_out_' + str(d_o) + '_Batch_n_' + str(b_n) + '_aug'
            
            print('Evaluating model: ', model_name)
            
            # Load the model
            # Set architecture
            opt = optimizers.Adam(lr=0.0001)
            model = galaxy_cnn((160, 160, 3), 37, d_o, b_n, d_s)
            model.load_weights(path_data + 'weights/' + model_name +'.h5')
            model.compile(loss='mean_squared_error', optimizer=opt, metrics=[rmse])
            
            # Evaluate model
            evaluation = model.evaluate_generator(generator(path_data + 'preprocess_img_test', labels_test, batch_size=batch_size), steps=steps)

            model_names.append(model_name)
            evaluations.append(evaluation)
            
            K.clear_session()

Evaluating model:  model_Dense_1024_Drop_out_True_Batch_n_False_aug
Evaluating model:  model_Dense_1024_Drop_out_True_Batch_n_True_aug
Evaluating model:  model_Dense_2048_Drop_out_True_Batch_n_False_aug
Evaluating model:  model_Dense_2048_Drop_out_True_Batch_n_True_aug


In [14]:
evaluations = np.array(evaluations)

In [15]:
rmse_list = evaluations[:, 1]
list_index = rmse_list.argsort()

In [16]:
for item in list_index:
    print(model_names[item], rmse_list[item])

model_Dense_2048_Drop_out_True_Batch_n_False_aug 0.093947293147561
model_Dense_1024_Drop_out_True_Batch_n_False_aug 0.09483851098556892
model_Dense_2048_Drop_out_True_Batch_n_True_aug 0.09487932226203578
model_Dense_1024_Drop_out_True_Batch_n_True_aug 0.09622151068429417
